### Import

In [ ]:
!pip install ipyplot -q

import pandas as pd
import numpy as np
import ipyplot
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

path = '../input/hpa-single-cell-image-classification/'
df_train = pd.read_csv(path + 'train.csv')

df_sub = pd.read_csv(path + 'sample_submission.csv')


colours = ['_red.png', '_green.png', '_blue.png', '_yellow.png']
TRAIN_PATHS = '/kaggle/input/hpa-single-cell-image-classification/train'
train_paths = [[os.path.join(TRAIN_PATHS, df_train.iloc[idx,0]) + colour for colour in colours] for idx in range(len(df_train))]
train_paths[1]

### EDA

In [ ]:
l_dict = {
0:  "Nucleoplasm", 
1:  "Nuclear Membrane",   
2:  "Nucleoli",   
3:  "Nucleoli Fibrillar Center" ,  
4:  "Nuclear Speckles",
5:  "Nuclear Bodies",
6:  "Endoplasmic Reticulum",   
7:  "Golgi Apparatus",
8:  "Intermediate Filaments",
9:  "Actin Filaments", 
10: "Microtubules",
11:  "Mitotic Spindle",
12:  "Centrosome",   
13:  "Plasma Membrane",
14:  "Mitochondria",   
15:  "Aggresome",
16:  "Cytosol",   
17:  "Vesicles and Punctate Cytosolic Patterns",   
18:  "Negative"
}

In [ ]:
df_train.Label.value_counts()[:50].sort_values().plot.barh(figsize=(10,10), title="Raw Label Count")

In [ ]:
label_counts = df_train.Label.str.split("|").explode().astype(int).value_counts().sort_values().rename(index=l_dict)
label_counts.plot.barh(figsize=(10,10), title="Individual Label Counts")

In [ ]:
label_counts.plot.pie(figsize=(10,10), title="Individual Label Distribution", ylabel="", autopct='%1.1f%%', fontsize=10, startangle=0)

In [ ]:
df_train['Label Count'] = df_train.Label.str.split("|").str.len()
df_train['Label Count'].value_counts().plot.bar(title="Label Length Count")

In [ ]:
from itertools import product

label_list = df_train.Label.str.split("|").to_list()

items = [map(int, a) for l in label_list for a in list(product(l, l))]

dc_df = pd.DataFrame(items, columns = ['LabelA', 'LabelB']) 
cmatrix = pd.crosstab(dc_df.LabelA, dc_df.LabelB)
np.fill_diagonal(cmatrix.values, 0)

f = plt.figure(figsize=(10, 10))
sns.heatmap(cmatrix, cmap="Blues")
plt.title("Label Co-occurrence")

### Visual Inspection

In [ ]:
def read_imgs(paths):
    return [plt.imread(a) for a in paths]
    
def rgby2rgb(rgby_arr):
    rgby_arr = np.dstack(np.array(rgby_arr))
    rgb_mat = np.zeros_like(rgby_arr)
    
    rgb_mat[:,:,0] = rgby_arr[:,:,0] + rgby_arr[:,:,3]
    rgb_mat[:,:,1] = rgby_arr[:,:,1] + rgby_arr[:,:,3]/2
    rgb_mat[:,:,2] = rgby_arr[:,:,2]
    
    return rgb_mat[:,:,:3]

In [ ]:
for entry in range(0, 19):    
    num = 10
    if entry == 11:
        continue 
        
    try:
        #Find Separate Channel Iamge Files per class
        i_sample = df_train[df_train.Label == str(entry)].iloc[:num]
    except:
        continue

    fig, axs = plt.subplots(1, num, figsize =(30,30))

    train_imgs = [os.path.join(TRAIN_PATHS, ex[1].ID) + "_green.png" for ex in i_sample.iterrows()]
    img_list = read_imgs(train_imgs)

    #Display individual RGBY channels
    for item_i in range(num):
        if item_i == len(img_list):
            break
        img_mat = np.zeros(list(img_list[item_i].shape) + [3])
        img_mat[:,:,1] = img_list[item_i]
        axs[item_i].imshow(img_mat)

    plt.suptitle(l_dict[entry], fontsize=16, y=0.55)
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
    plt.show()

In [ ]:
titles = ['Microtubules','Proteins of Interest', 'Nuclei', 'Endoplasmic Reticulums', 'Blended']

for entry in range(0, 19):    
    for i in range(0, 3):        
        try:
            #Find Separate Channel Iamge Files per class
            i_sample = df_train[df_train.Label == str(entry)].iloc[i]
        except:
            continue

        fig, axs = plt.subplots(1, 5, figsize =(30,30))

        train_imgs = [os.path.join(TRAIN_PATHS, i_sample.ID)+ colour for colour in colours]
        img_list = read_imgs(train_imgs)

        #Display individual RGBY channels
        for channel in range(4):
            img_mat = np.zeros(list(img_list[channel].shape) + [3])

            if channel != 3:
                img_mat[:,:,channel] = img_list[channel]
            else:
                img_mat[:,:,0] = img_list[channel] 
                img_mat[:,:,1] = img_list[channel]

            axs[channel].imshow(img_mat)

        #Display blended image
        blended_img = rgby2rgb(img_list)
        axs[4].imshow(blended_img)

        plt.suptitle(l_dict[int(i_sample.Label)], fontsize=16, y=0.6)

        #Set subplot titles
        for ti, t in enumerate(titles):
            axs[ti].set_title(t)

        plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
        plt.show()

### Datset Prep
The rest of the notebook will require TPU support.

In [ ]:
!pip install efficientnet -q
import tensorflow as tf
import efficientnet.tfkeras as efn

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='png'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

In [ ]:
label2id_dict = {
 'Nucleoplasm': 0,
 'Nuclear Membrane': 1,
 'Nucleoli': 2,
 'Nucleoli Fibrillar Center': 3,
 'Nuclear Speckles': 4,
 'Nuclear Bodies': 5,
 'Endoplasmic Reticulum': 6,
 'Golgi Apparatus': 7,
 'Intermediate Filaments': 8,
 'Actin Filaments': 9,
 'Microtubules': 10,
 'Mitotic Spindle': 11,
 'Centrosome': 12,
 'Plasma Membrane': 13,
 'Mitochondria': 14,
 'Aggresome': 15,
 'Cytosol': 16,
 'Vesicles': 17,
 'Negative': 18
}

load_dir = "gs://green_channels/"

#Preprocessing Dataset 
df = pd.read_csv('../input/classification-label-csv-green/df_green.csv')
label_cols = df.columns[2:21]
paths = load_dir + df['ID']
labels = df[label_cols].values

# df['label_count'] = df.Label.str.split("|").str.len()
# df = df[df.label_count == 1]
# df['label_name'] = df["Label"].apply(lambda x: l_dict[int(x)])

df['path'] = df["ID"].apply(lambda x: load_dir + x + "")

In [ ]:
test_indices = []
for i in range(0, 19):
    idx_list = df[df["Label"] == str(i)].sample(frac=.2).index
    test_indices.extend(list(idx_list))

training_df = df.loc[~df.index.isin(test_indices)]
test_df = df.loc[test_indices]

valid_indices = []
for i in range(0, 19):
    idx_list = training_df[training_df["Label"] == str(i)].sample(frac=.2).index
    valid_indices.extend(list(idx_list))

valid_df = training_df.loc[valid_indices]
training_df = training_df.loc[~training_df.index.isin(valid_indices)]

label_cols = df.columns[2:21]

training_paths = load_dir + training_df['ID']
training_labels = training_df[label_cols].values

valid_paths = load_dir + valid_df['ID']
valid_labels = valid_df[label_cols].values

test_paths = load_dir + test_df['ID']
test_labels = test_df[label_cols].values


# training_df["img_mean"] = training_df["path"].apply(lambda x: plt.imread(x).ravel().mean())
# valid_df["img_mean"] = valid_df["path"].apply(lambda x: plt.imread(x).ravel().mean())

# training_df.to_csv("training_df.csv", index=False)
# valid_df.to_csv("valid_df.csv", index=False)
# test_df.to_csv("test_df.csv", index=False)

In [ ]:
def filter_95_percent(target_df):
    refined_training_df = []
    target_df["Label"] = target_df["Label"].astype(str)
    for i in range(0, 19):
        filtered_df = target_df[target_df["Label"].str.contains(str(i))]
        mean_stats = filtered_df.describe()["img_mean"]

        max_std = mean_stats["mean"] + (2 * mean_stats["std"])
        min_std = mean_stats["mean"] - (2 * mean_stats["std"])

        id_list = filtered_df[(filtered_df["img_mean"] >= min_std) & (filtered_df["img_mean"] <= max_std)].index 
        refined_training_df.extend(list(id_list))

    refined_training_df = target_df.loc[set(refined_training_df)]
    
    return refined_training_df

In [ ]:
label_cols = df.columns[2:21]

file_format = ""

training_df = pd.read_csv("../input/dataset-df/training_df.csv")
train_paths = load_dir + training_df['ID'] + file_format
training_df["path"] = train_paths
train_labels = training_df[label_cols].values

valid_df = pd.read_csv("../input/dataset-df/valid_df.csv")
valid_paths = load_dir + valid_df['ID'] + file_format
valid_df["path"] = valid_paths
valid_labels = valid_df[label_cols].values

test_df = pd.read_csv("../input/dataset-df/test_df.csv")
test_paths = load_dir + test_df['ID'] + file_format
test_df["path"] = test_paths
test_labels = test_df[label_cols].values

sample_df = df.sample(frac=0.01)
sample_paths = load_dir + sample_df['ID'] + file_format
sample_labels = sample_df[label_cols].values

refined_training_df = filter_95_percent(training_df)
train_paths = load_dir + refined_training_df['ID'] + file_format
train_labels = refined_training_df[label_cols].values

refined_valid_df = filter_95_percent(valid_df)
valid_paths = load_dir + refined_valid_df['ID'] + file_format
valid_labels = refined_valid_df[label_cols].values

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

IMSIZE = (224, 600)
IMS = 1

decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]))
test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder, repeat=True, augment=True
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder, 
    repeat=False, shuffle=False, augment=False
)
    
test_dataset = build_dataset(
    test_paths, test_labels, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=decoder
)

sample_dataset = build_dataset(
    sample_paths, sample_labels, bsize=BATCH_SIZE, decode_fn=decoder, 
    repeat=False, shuffle=False, augment=False
)

### EfficientNet Training

In [ ]:
try:
    n_labels = train_labels.shape[1]
except:
    n_labels = 1
    
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(n_labels, activation='sigmoid')
    ])
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(multi_label=True)])
        
    model.summary()
    
# model_path = "model_path"
# model = tf.keras.models.load_model(model_path)

In [ ]:
class_weight = {}
img_count = len(training_df)

for i in range(0, 19):
    count_i = training_df[str(i)].value_counts()[1]
    weight = 1 - count_i / img_count
    class_weight[i] = weight
    
class_weight

In [ ]:
steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'prefileter_CE.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3, min_lr=1e-6, mode='auto')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    train_dataset, 
    epochs=20,
    verbose=1,
    callbacks = [checkpoint, lr_reducer, early_stopping],
    steps_per_epoch=steps_per_epoch,
    class_weight = class_weight,
    validation_data=valid_dataset
)

### Performance Evaluation

In [ ]:
import seaborn as sn
from sklearn.metrics import classification_report, confusion_matrix

prediction_probs = model.predict(test_dataset, verbose=1)
prediction_classes = np.argmax(prediction_probs, axis=-1)

In [ ]:
y_true = list(test_df["Label"])
y_true = list(map(lambda x: int(x), y_true))

cmat = confusion_matrix(y_true, prediction_classes)
figure = plt.figure(figsize=(12,12))
sn.heatmap(cmat,annot=True, fmt='')
print(classification_report(y_true, prediction_classes))

### Color Histogram Distribution

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from collections import defaultdict
from tqdm import tqdm 

for img_path in tqdm(list(training_df["path"])[:30]):
    img = plt.imread(img_path,0)
    img_ravel = img.ravel()
    hist, bins = np.histogram(img.ravel(),256,[0,256])    
    plt.hist(img_ravel, 10)
    plt.axvline(img_ravel.mean(), color='k', linestyle='dashed', linewidth=1)
    plt.show()
    
    plt.imshow(img)
    plt.show()

In [ ]:
from tqdm import tqdm 

min_val = 0.00
max_val = 0.3
path_list = list(training_df[(training_df["img_mean"] <= min_val) | (training_df["img_mean"] >= max_val)] ["path"])
print(len(path_list))

for idx, img_path in tqdm(enumerate(path_list)):
    img = plt.imread(img_path,0)
    plt.imshow(img)
    plt.show()
    if idx == 20:
        break

In [ ]:
from sklearn.manifold import TSNE
from collections import defaultdict

hist_embedded = TSNE(n_components=2).fit_transform(hist_list)

hist_dict = defaultdict(list)

fig = plt.figure(figsize=(20,20))

for idx, i in enumerate(list(training_df["Label"][:100])):
    hist_dict[int(i)].append(hist_embedded[idx])
    
img_paths = list(training_df["path"])[:100]

for i in range(0, 19):
    points = np.array(hist_dict[i])
    if len(points):
        plt.scatter(points[:,0], points[:,1])

### Image Clustering

In [ ]:
try:
    n_labels = train_labels.shape[1]
except:
    n_labels = 1
    
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics=["categorical_accuracy"])
        
    model.summary()

In [ ]:
X_raw = model.predict(sample_dataset)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from collections import defaultdict

X_embedded = TSNE(n_components=2).fit_transform(X_raw)

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

def getImage(path):
    return OffsetImage(plt.imread(path), zoom=0.02)

In [ ]:
cat_dict = defaultdict(list)

fig = plt.figure(figsize=(5,5))
for idx, i in enumerate(list(sample_df["Label"])):
    cat_dict[int(i)].append(X_embedded[idx])
    
counter = 0
img_paths = list(df["path"])

for i in range(0, 19):
    points = np.array(cat_dict[i])
    if len(points):
        plt.scatter(points[:,0], points[:,1])
        
plt.title("EfficientNet B7 Embeddings - t-SNE")

###  SIFT-BOVW Representation

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

def load_images_from_folder(paths):
    images = []
    for p in paths:
        img = cv.imread(p,0) 
        images.append(img)
    return images

images = load_images_from_folder(sample_df["path"])

In [ ]:
from tqdm import tqdm 

def sift_features(images):
    sift_vectors = []
    descriptor_list = []
    sift = cv.SIFT_create()
    keypt_list = []
    for img in tqdm(images):
        kp, des = sift.detectAndCompute(img,None)
        
        for d in des:
            descriptor_list.append(d)
        
        sift_vectors.append(des)
        keypt_list.append(kp)
    return descriptor_list, sift_vectors, keypt_list

descriptor_list, desc_vectors, keypt_vectors = sift_features(images) 

In [ ]:
from sklearn.cluster import MiniBatchKMeans

k = 100
batch_size = len(desc_vectors) * 3
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=0).fit(descriptor_list)

In [ ]:
kmeans.verbose = False

histo_list = []

for kp, des in zip(keypt_vectors, desc_vectors):
    histo = np.zeros(k)
    nkp = np.size(kp)

    for d in des:
        idx = kmeans.predict([d])
        histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

    histo_list.append(histo)

In [ ]:
from sklearn.manifold import TSNE

hist_tsne = TSNE(n_components=2).fit_transform(histo_list)

In [ ]:
from collections import defaultdict

bovw_dict = defaultdict(list)
fig = plt.figure(figsize=(5,5))

for idx, i in enumerate(list(sample_df["Label"])):
    bovw_dict[int(i)].append(hist_tsne[idx])
    
for i in range(0, 19):
    points = np.array(bovw_dict[i])
    if len(points):
        plt.scatter(points[:,0], points[:,1])

plt.title("SIFT-BOVW - t-SNE")

### Fine-tuned EfficientNet Embeddings

In [ ]:
from keras.models import Model

model_path = "../input/hpa-classification-efnb7-train/model_green.h5"
model = tf.keras.models.load_model(model_path)

In [ ]:
model= Model(inputs=model.input, outputs=model.layers[-1].output)

In [ ]:
decoder = build_decoder(with_labels=True, target_size=(600, 600))
sample_dataset = build_dataset(
    sample_paths, sample_labels, bsize=BATCH_SIZE, decode_fn=decoder, 
    repeat=False, shuffle=False, augment=False
)

In [ ]:
fine_tuned_embeddings = model.predict(sample_dataset)
fine_tsne = TSNE(n_components=2).fit_transform(fine_tuned_embeddings)

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

def getImage(path):
    return OffsetImage(plt.imread(path), zoom=0.02)

In [ ]:
fine_tsne_dict = defaultdict(list)
fig = plt.figure(figsize=(5,5))

for idx, i in enumerate(list(sample_df["Label"])):
    fine_tsne_dict[int(i)].append(fine_tsne[idx])
        
counter = 0
img_paths = list(sample_df["path"])

for i in range(0, 19):
    points = np.array(fine_tsne_dict[i])
    if len(points):
        plt.scatter(points[:,0], points[:,1])

plt.title("Fine-tuned EfficientNet Embeddings - t-SNE")

### U-Net

In [ ]:
!pip install keras-unet

In [ ]:
from keras_unet.models import vanilla_unet
from keras.models import Model
from sklearn.manifold import TSNE

decoder = build_decoder(with_labels=True, target_size=(240, 240))
sample_dataset = build_dataset(
    sample_paths, sample_labels, bsize=BATCH_SIZE, decode_fn=decoder, 
    repeat=False, shuffle=False, augment=False
)

model = vanilla_unet(input_shape=(240, 240, 3))
model_layer = Model(inputs=model.input, outputs=model.layers[-1].output)
model = tf.keras.Sequential([
        model_layer,
        tf.keras.layers.GlobalMaxPooling2D()
    ])

unet_embeddings = model.predict(sample_dataset)
unet_tsne = TSNE(n_components=2).fit_transform(unet_embeddings)

In [ ]:
fig = plt.figure(figsize=(5,5))

unet_tsne_dict = defaultdict(list)

for idx, i in enumerate(list(sample_df["Label"])):
    unet_tsne_dict[int(i)].append(unet_tsne[idx])
    
for i in range(0, 19):
    points = np.array(unet_tsne_dict[i])
    if len(points):
        plt.scatter(points[:,0], points[:,1])
        
plt.title("UNet Embeddings - t-SNE")

### Individual Cell Segmentation

In [ ]:
!pip install -q "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install -q "../input/hpapytorchzoozip/pytorch_zoo-master"
!pip install -q "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell

single_label_images = df_train[df_train["Label Count"] == 1]

In [ ]:
def format4segmentator(train_paths, n=4):
    """
    restructures the image paths into a list of lists
    """
    zipped = list(map(list, zip(*train_paths[:n])))
    return [zipped[0], zipped[3], zipped[2]]
    
sample_list = format4segmentator(train_paths)
sample_list

In [ ]:
segmentator = cellsegmentator.CellSegmentator(
    "./nuclei-model.pth",
    "./cell-model.pth",
    scale_factor=0.25,
    device="cuda",
    padding=False,
    multi_channel_model=True,
)

# For nuclei
nuc_segmentations = segmentator.pred_nuclei(sample_list[2])

# For full cells
cell_segmentations = segmentator.pred_cells(sample_list)

# post-processing
nuclei_mask = label_nuclei(nuc_segmentations[0])
nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])

In [ ]:
#Show the original image
img = rgby2rgb(read_imgs(train_paths[0]))
plt.imshow(img)

In [ ]:
#Calculates the cell contours
contours, hierarchy= cv2.findContours(cell_mask.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
#Plot individual cells
for i in x:
    cnt = contours[i]
    ix,yc,w,h = cv2.boundingRect(cnt)

    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(poi[yc:yc+h, ix:ix+w])
    plt.title('Cell Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(cell_mask[yc:yc+h, ix:ix+w])
    plt.title('Cell Mask')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(poi[yc:yc+h, ix:ix+w])
    plt.imshow(cell_mask[yc:yc+h, ix:ix+w], alpha=0.6)
    plt.title('Cell Image + Mask')
    plt.axis('off')
    plt.show()